# Sodoku Quality Test 

In [10]:
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

In [11]:
path = "../sudoku_data/"
data = pd.read_csv(path+"sudoku_1_mio_sequenced_columns.csv")
data = data[:10]
display(data)

,Q1A,Q1B,Q1C,Q1D,Q1E,Q1F,Q1G,Q1H,Q1I,Q2A,...,S8I,S9A,S9B,S9C,S9D,S9E,S9F,S9G,S9H,S9I
0,0,0,4,3,0,0,2,0,9,0,...,4,5,4,2,9,1,6,3,7,8
1,0,4,0,1,0,0,0,5,0,1,...,2,2,5,4,3,9,7,1,8,6
2,6,0,0,1,2,0,3,8,4,0,...,3,5,6,2,3,4,8,1,9,7
3,4,9,7,2,0,0,0,0,0,1,...,4,5,3,4,8,9,7,2,6,1
4,0,0,5,9,1,0,3,0,8,0,...,3,8,9,1,7,3,6,4,2,5
5,1,0,0,0,0,5,0,0,7,3,...,8,9,7,8,1,4,6,3,5,2
6,0,0,9,0,6,5,4,3,0,0,...,4,1,7,8,2,4,6,3,9,5
7,0,0,0,0,0,0,6,5,7,7,...,1,4,3,5,9,1,8,2,7,6
8,5,0,3,0,7,0,1,9,0,0,...,7,1,8,6,7,2,3,9,4,5
9,0,6,0,7,2,0,9,0,8,0,...,2,2,1,5,8,7,6,4,9,3


## All cells are filled => no Null values

In [12]:
data = data[~data.isnull().any(axis=1)]

## All numbers must be between 0 and 9 for the quizzes 

In [13]:
query_conditions = []
# Iterate over all quizzes col names 
for col_name in data.columns[:81]:
    query_conditions.append (col_name + " >= 0 & " + col_name+ " <= 9")
data = data.query(" & ".join(query_conditions))

## Solutions must contain numbers between 1 and 9

In [14]:
query_conditions = []
# Iterate over all col names with solutions 
for col_name in data.columns[81:]:
    query_conditions.append (col_name + " > 0 & " + col_name + " <=9")
data = data.query(" & ".join(query_conditions))

## All cells != 0 must have identical content

In [16]:
# Build Query
query_conditions = [] 
num_cols = int(data.columns.size/2 )
for i in range(num_cols):
    query_conditions.append ("("+data.columns[i]+ " == 0 | " + data.columns[i] +" == "+data.columns[i+num_cols]+")")
data = data.query(" & ".join(query_conditions))
data

,Q1A,Q1B,Q1C,Q1D,Q1E,Q1F,Q1G,Q1H,Q1I,Q2A,...,S8I,S9A,S9B,S9C,S9D,S9E,S9F,S9G,S9H,S9I
0,0,0,4,3,0,0,2,0,9,0,...,4,5,4,2,9,1,6,3,7,8
1,0,4,0,1,0,0,0,5,0,1,...,2,2,5,4,3,9,7,1,8,6
2,6,0,0,1,2,0,3,8,4,0,...,3,5,6,2,3,4,8,1,9,7
3,4,9,7,2,0,0,0,0,0,1,...,4,5,3,4,8,9,7,2,6,1
4,0,0,5,9,1,0,3,0,8,0,...,3,8,9,1,7,3,6,4,2,5
5,1,0,0,0,0,5,0,0,7,3,...,8,9,7,8,1,4,6,3,5,2
6,0,0,9,0,6,5,4,3,0,0,...,4,1,7,8,2,4,6,3,9,5
7,0,0,0,0,0,0,6,5,7,7,...,1,4,3,5,9,1,8,2,7,6
8,5,0,3,0,7,0,1,9,0,0,...,7,1,8,6,7,2,3,9,4,5
9,0,6,0,7,2,0,9,0,8,0,...,2,2,1,5,8,7,6,4,9,3


## Checking Solutions: Checking all Rows and Columns

In [18]:
# Checking each row
#start = 81
df_only_solutions = data[data.columns[81:]]
display(df_only_solutions)

def is_in_range_and_unique(values:list, min_val:int, max_val:int)-> bool:
    """Check if the list given contains only unique elements"""
    if len(values) == len(set(values)):
        return min(values) == min_val and max(values)== max_val 
    return False

def check_unique_num_in_row_and_col(row_as_series: pd.Series) -> bool:
    """Input: pd.Series, Output boolean"""
    for start in range(0,row_as_series.size,9):
        if not is_in_range_and_unique (row_as_series[start:start+9].to_list(),1,9):
            return False
    for start in range(0,9):
        list_of_indexes = [*range (start,row_as_series.size,9)]
        if not is_in_range_and_unique (row_as_series.iloc[list_of_indexes].to_list(),1,9):
            return False
    return True

data["test_row"] = df_only_solutions.apply(lambda x: check_unique_num_in_row_and_col(x),axis=1)
data = data.query("test_row == True")
data = data.drop(columns=["test_row"])
data
       

,S1A,S1B,S1C,S1D,S1E,S1F,S1G,S1H,S1I,S2A,...,S8I,S9A,S9B,S9C,S9D,S9E,S9F,S9G,S9H,S9I
0,8,6,4,3,7,1,2,5,9,3,...,4,5,4,2,9,1,6,3,7,8
1,3,4,6,1,7,9,2,5,8,1,...,2,2,5,4,3,9,7,1,8,6
2,6,9,5,1,2,7,3,8,4,1,...,3,5,6,2,3,4,8,1,9,7
3,4,9,7,2,5,8,3,1,6,1,...,4,5,3,4,8,9,7,2,6,1
4,4,6,5,9,1,2,3,7,8,1,...,3,8,9,1,7,3,6,4,2,5
5,1,9,4,6,8,5,2,3,7,3,...,8,9,7,8,1,4,6,3,5,2
6,2,8,9,7,6,5,4,3,1,3,...,4,1,7,8,2,4,6,3,9,5
7,8,9,4,2,3,1,6,5,7,7,...,1,4,3,5,9,1,8,2,7,6
8,5,6,3,4,7,2,1,9,8,2,...,7,1,8,6,7,2,3,9,4,5
9,1,6,3,7,2,5,9,4,8,5,...,2,2,1,5,8,7,6,4,9,3


,Q1A,Q1B,Q1C,Q1D,Q1E,Q1F,Q1G,Q1H,Q1I,Q2A,...,S8I,S9A,S9B,S9C,S9D,S9E,S9F,S9G,S9H,S9I
0,0,0,4,3,0,0,2,0,9,0,...,4,5,4,2,9,1,6,3,7,8
1,0,4,0,1,0,0,0,5,0,1,...,2,2,5,4,3,9,7,1,8,6
2,6,0,0,1,2,0,3,8,4,0,...,3,5,6,2,3,4,8,1,9,7
3,4,9,7,2,0,0,0,0,0,1,...,4,5,3,4,8,9,7,2,6,1
4,0,0,5,9,1,0,3,0,8,0,...,3,8,9,1,7,3,6,4,2,5
5,1,0,0,0,0,5,0,0,7,3,...,8,9,7,8,1,4,6,3,5,2
6,0,0,9,0,6,5,4,3,0,0,...,4,1,7,8,2,4,6,3,9,5
7,0,0,0,0,0,0,6,5,7,7,...,1,4,3,5,9,1,8,2,7,6
8,5,0,3,0,7,0,1,9,0,0,...,7,1,8,6,7,2,3,9,4,5
9,0,6,0,7,2,0,9,0,8,0,...,2,2,1,5,8,7,6,4,9,3


## Checking Solutions: Checking all Squares

In [9]:
# check square
# there are nine 3x3 squares 
# the first square has the indexes (S1A, S1B, S1C, 
#                                   S2A, S2B, S2C,
#                                   S3A, S3B, S3C)

def get_square_definition ():
    """Returns the square positions of a 9*9 sodoku"""
    col_names = ["A","B","C","D","E","F","G","H","I"] 
    row_nums = [*range(1,10)]

    col_chunks = np.array_split(col_names, 3)
    row_chunks = np.array_split(row_nums, 3)
    square_pos=[]
    for curr_rows in row_chunks :
        for curr_cols in col_chunks:
            pos_names = []        
            for curr_row in curr_rows :
                for curr_col in curr_cols:
                    pos_names.append("S"+str(curr_row)+str(curr_col))
            square_pos.append(pos_names)
    return square_pos       

def check_squares(row_as_series:pd.Series) -> bool:
    """Input=pd.Series Output = True/False"""
    for curr_square in get_square_definition() : 
        if not is_in_range_and_unique(row_as_series.get(curr_square).to_list(),1,9):
            return False
    return True
    

df_only_solutions = data[data.columns[81:]]
#df_only_solutions.apply(lambda x: check_squares(x),axis=1)
data["test_row"] = df_only_solutions.apply(lambda x: check_squares(x),axis=1)
data = data.query("test_row == True")
data = data.drop(columns=["test_row"])
data

,Q1A,Q1B,Q1C,Q1D,Q1E,Q1F,Q1G,Q1H,Q1I,Q2A,...,S8I,S9A,S9B,S9C,S9D,S9E,S9F,S9G,S9H,S9I
0,0,0,4,3,0,0,2,0,9,0,...,4,5,4,2,9,1,6,3,7,8
1,0,4,0,1,0,0,0,5,0,1,...,2,2,5,4,3,9,7,1,8,6
2,6,0,0,1,2,0,3,8,4,0,...,3,5,6,2,3,4,8,1,9,7
3,4,9,7,2,0,0,0,0,0,1,...,4,5,3,4,8,9,7,2,6,1
4,0,0,5,9,1,0,3,0,8,0,...,3,8,9,1,7,3,6,4,2,5
5,1,0,0,0,0,5,0,0,7,3,...,8,9,7,8,1,4,6,3,5,2
6,0,0,9,0,6,5,4,3,0,0,...,4,1,7,8,2,4,6,3,9,5
7,0,0,0,0,0,0,6,5,7,7,...,1,4,3,5,9,1,8,2,7,6
8,5,0,3,0,7,0,1,9,0,0,...,7,1,8,6,7,2,3,9,4,5
9,0,6,0,7,2,0,9,0,8,0,...,2,2,1,5,8,7,6,4,9,3
